In [1]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPool2D , Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

2022-01-09 23:24:55.864497: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.


In [2]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [3]:
print(tf.__version__)

2.4.1


In [4]:

tf.config.list_physical_devices('GPU')
#print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

2022-01-09 23:24:56.919659: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-09 23:24:56.920228: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-09 23:24:56.974053: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 23:24:56.975622: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-09 23:24:56.975687: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-09 23:24:56.981214: I tensorflow/stream_executor/platform/defau

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

rm/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-09 23:24:56.983718: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-01-09 23:24:56.984260: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2022-01-09 23:24:56.989417: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusolver.so.10
2022-01-09 23:24:56.990490: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusparse.so.11
2022-01-09 23:24:56.990748: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-01-09 23:24:56.990913: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so r

In [5]:
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

GPUs:  1


In [6]:
train = pd.read_csv('train_metadata.csv')
test = pd.read_csv('test_metadata.csv')
#submission = pd.read_csv('sample_submission.csv')

train_dir = 'train/'
test_dir = 'test/'

In [7]:
label = []
train_data = []
for i in range(train.shape[0]):
    train_data.append(train_dir + train['image_name'].iloc[i] + '.jpg')
    label.append(train['diagnosis'].iloc[i])
    
test_data = []
for i in range(test.shape[0]):
    test_data.append(test_dir + test['image'].iloc[i] + '.jpg')

In [8]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
label_df = pd.DataFrame(label, columns = ['diagnosis'])
one_hot_encoder.fit(label_df)

label_df_encoded = one_hot_encoder.transform(label_df)
label_df_encoded = pd.DataFrame(data=label_df_encoded, columns=one_hot_encoder.categories_)
print(label_df_encoded.columns)

MultiIndex([('atypical melanocytic proliferation',),
            (               'cafe-au-lait macule',),
            (                       'lentigo NOS',),
            (               'lichenoid keratosis',),
            (                          'melanoma',),
            (                             'nevus',),
            (              'seborrheic keratosis',),
            (                     'solar lentigo',),
            (                           'unknown',)],
           )


In [9]:
df_train = pd.DataFrame(train_data)
df_train.columns = ['images']
df_train = pd.concat([df_train, label_df_encoded], axis = 1)
print(df_train.columns)
df_test = pd.DataFrame(test_data)
df_test.columns = ['images']

Index([                               'images',
       ('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_train.iloc[:, 0], df_train.iloc[:,1::], test_size = 0.3, random_state = 42)

train = pd.DataFrame(X_train)
train.columns = ['images']
train = pd.concat([train, y_train], axis = 1)

test = pd.DataFrame(X_test)
test.columns = ['images']
test = pd.concat([test, y_test], axis = 1)

In [15]:
train.columns[1:]

Index([('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=50,
                                   zoom_range = 0.3,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   horizontal_flip=True,
                                   fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col=train.columns[0],
    y_col=train.columns[1:],
    target_size=(224, 224),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    test,
    x_col=test.columns[0],
    y_col=test.columns[1:],
    target_size=(224, 224),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

Found 23188 validated image filenames.
Found 9938 validated image filenames.


In [12]:
from tensorflow.keras.applications import vgg16, vgg19
from tensorflow.keras.models import Model
input_shape = (224,224,3)
vgg = vgg19.VGG19(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)

vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False
    
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

2022-01-09 23:25:11.499964: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-09 23:25:11.500695: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-09 23:25:11.500929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 23:25:11.501976: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-09 23:25

80142336/80134624 [==============================] - 24s 0us/step


/tmp/ipykernel_11665/932173164.py:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f728ec91f40>,input_1,False
1,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f728003fbb0>,block1_conv1,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f728021edc0>,block1_conv2,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f728003ffd0>,block1_pool,False
4,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f726f34ec10>,block2_conv1,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f726f342070>,block2_conv2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f72801d1850>,block2_pool,False
7,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f72801d9f70>,block3_conv1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f72801d1130>,block3_conv2,False
9,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f72801e04c0>,block3_conv3,False


In [13]:
vgg_model.trainable = True

set_trainable = False
for layer in vgg_model.layers:
    if layer.name in ['block5_conv1', 'block4_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])    

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f728ec91f40>,input_1,False
1,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f728003fbb0>,block1_conv1,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f728021edc0>,block1_conv2,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f728003ffd0>,block1_pool,False
4,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f726f34ec10>,block2_conv1,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f726f342070>,block2_conv2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f72801d1850>,block2_pool,False
7,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f72801d9f70>,block3_conv1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f72801d1130>,block3_conv2,False
9,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f72801e04c0>,block3_conv3,False


In [22]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
import tensorflow as tf

class_num = 9

model = Sequential()
model.add(vgg_model)
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(class_num, activation='softmax'))

opt = optimizers.Adam(lr=0.0001)
model.compile(loss="categorical_crossentropy", metrics= 'accuracy',optimizer=opt)

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 25088)             20024384  
_________________________________________________________________
dense_3 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 9)                 4617      
Total params: 33,137,225
Trainable params: 30,811,657
Non-trainable params: 2,325,568
__________________________________

In [23]:
batch_size = 128
epochs = 100
nb_train_steps = train.shape[0] // batch_size
nb_val_steps = test.shape[0] // batch_size

checkpoint = ModelCheckpoint("vgg19.h5", monitor='val_accuracy', verbose=5, save_best_only=True, save_weights_only=False, mode='auto')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')

model.fit_generator(train_generator, steps_per_epoch=nb_train_steps, epochs=epochs,
                              validation_data=test_generator, validation_steps=nb_val_steps, 
                              verbose=1, callbacks=[checkpoint,early])

Epoch 1/100
181/181 [==============================] - 80s 442ms/step - loss: 0.6828 - accuracy: 0.8545 - val_loss: 0.4263 - val_accuracy: 0.8929

Epoch 00001: val_accuracy improved from -inf to 0.89286, saving model to vgg19.h5
Epoch 2/100
181/181 [==============================] - 79s 437ms/step - loss: 0.3888 - accuracy: 0.9206 - val_loss: 0.6286 - val_accuracy: 0.9188

Epoch 00002: val_accuracy improved from 0.89286 to 0.91883, saving model to vgg19.h5
Epoch 3/100
181/181 [==============================] - 78s 430ms/step - loss: 0.3146 - accuracy: 0.9398 - val_loss: 0.4589 - val_accuracy: 0.9058

Epoch 00003: val_accuracy did not improve from 0.91883
Epoch 4/100
181/181 [==============================] - 79s 435ms/step - loss: 0.3252 - accuracy: 0.9280 - val_loss: 0.4096 - val_accuracy: 0.9107

Epoch 00004: val_accuracy did not improve from 0.91883
Epoch 5/100
181/181 [==============================] - 79s 439ms/step - loss: 0.3705 - accuracy: 0.9134 - val_loss: 0.3827 - val_accura

In [142]:
test_datagen = ImageDataGenerator(rescale=1./255)

data = test_datagen.flow_from_dataframe(dataframe = df_train.iloc[:100,:],
      x_col="images", class_mode=None, 
      batch_size=1, target_size=(224,224), shuffle=False)

predictions= model.predict_generator(data, max_queue_size=64, workers = 32, verbose=1)
print(predictions[:100,0])

Found 100 validated image filenames.
100/100 [==============================] - 2s 13ms/step
[4.02194164e-05 5.63539788e-06 2.81048415e-04 1.58005496e-04
 8.13154566e-06 1.15607054e-06 7.74794626e-06 8.45311952e-05
 2.77789150e-06 1.14574523e-05 2.88193296e-05 7.84816257e-06
 9.25553613e-05 1.26370387e-05 3.30652540e-07 2.35567626e-07
 1.59345087e-04 2.28879653e-05 3.07332193e-05 5.42199132e-06
 1.46865204e-05 1.85832487e-05 6.09921608e-06 1.12449598e-05
 2.58364616e-06 1.02557342e-05 5.06941986e-04 4.65411809e-04
 2.22622810e-04 3.26497902e-05 2.46258169e-05 1.29956597e-05
 1.68221202e-04 7.83744417e-06 1.30526578e-05 9.40173413e-06
 6.83918825e-06 1.79584786e-05 9.66640982e-07 9.46034015e-06
 1.05999361e-05 4.50492022e-04 3.72153096e-04 2.44434814e-05
 1.13449780e-04 3.37643678e-06 6.12634001e-04 1.19846145e-05
 1.33305817e-04 2.07957652e-04 2.34068866e-04 1.78789912e-06
 8.83722259e-06 7.77743175e-04 4.53867898e-07 1.13007109e-05
 4.22467429e-06 1.65791832e-07 8.47377123e-07 6.72364

In [144]:
print(predictions)

[[4.02194164e-05 1.00000187e-04 1.92573643e-03 6.67932967e-04
  2.96887662e-02 5.32075204e-02 4.61461814e-03 5.70069824e-04
  9.05913055e-01]
 [5.63539788e-06 1.73287681e-05 5.30908408e-04 1.40515345e-04
  1.11860922e-02 3.25741731e-02 2.22273543e-03 1.52355919e-04
  9.45892513e-01]
 [2.81048415e-04 3.80293612e-04 1.08293293e-03 1.78623700e-03
  1.58669632e-02 9.73227501e-01 1.75216061e-03 1.97760426e-04
  1.49566717e-02]
 [1.58005496e-04 1.75657115e-04 1.02323352e-03 1.77667814e-03
  8.73947144e-03 9.43202138e-01 2.26475950e-03 1.31764653e-04
  3.05724405e-02]
 [8.13154566e-06 2.33614028e-05 6.33902731e-04 1.56123278e-04
  1.18362550e-02 3.33268233e-02 2.57141585e-03 1.85633631e-04
  9.43630278e-01]
 [1.15607054e-06 3.15677812e-06 1.74318804e-04 3.54986551e-05
  2.86310026e-03 2.99557578e-02 8.43638962e-04 6.34322932e-05
  9.63136613e-01]
 [7.74794626e-06 1.73326025e-05 6.63306506e-04 1.27361796e-04
  1.07218828e-02 3.50216515e-02 2.30921595e-03 1.58894472e-04
  9.43458974e-01]
 [8.45

In [145]:
train.columns

Index([                               'images',
       ('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')